If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [1]:
!pip install -q datasets evaluate transformers rouge-score nltk transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.8 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.31.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [XSum dataset](https://arxiv.org/pdf/1808.08745.pdf) (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

![Widget inference on a summarization task](https://github.com/huggingface/notebooks/blob/main/examples/images/summarization.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [6]:
model_checkpoint = "facebook/bart-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [7]:
!wget https://raw.githubusercontent.com/GuyRobot/AINotesBook/main/EHealthChatDataset.json

--2023-08-04 03:18:41--  https://raw.githubusercontent.com/GuyRobot/AINotesBook/main/EHealthChatDataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7686023 (7.3M) [text/plain]
Saving to: ‘EHealthChatDataset.json’

EHealthChatDataset. 100%[===================>]   7.33M  --.-KB/s    in 0.04s   

2023-08-04 03:18:47 (182 MB/s) - ‘EHealthChatDataset.json’ saved [7686023/7686023]



In [8]:
!pip install -q nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00


In [9]:
import datasets
from datasets import load_dataset
from evaluate import load

train_ds = load_dataset("json", data_files=["EHealthChatDataset.json"], split="train[:80%]")
vals_ds = load_dataset("json", data_files=["EHealthChatDataset.json"], split="train[80%:]")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
from datasets import DatasetDict
raw_datasets = DatasetDict()
raw_datasets["train"] = train_ds
raw_datasets["validation"] = vals_ds
# raw_datasets = load_dataset("xsum")
# raw_datasets = load_dataset("json", data_files={"train": "ehealthforumQAs.json", "validation": "ehealthforumQAs.json"})
metric = load("rouge")

In [11]:
# import nlp
# dataset_cc = nlp.concatenate_datasets([train_ds, vals_ds])

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'url', 'tags'],
        num_rows: 4006
    })
    validation: Dataset({
        features: ['question', 'answer', 'url', 'tags'],
        num_rows: 1002
    })
})

To access an actual element, you need to select a split first, then give an index:

In [13]:
raw_datasets["train"][0]

{'question': 'how do i stop smoking now',
 'answer': 'stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks',
 'url': 'http://ehealthforum.com/health/stop-smoking-question-t462882.html',
 'tags': ['addiction', 'stop smoking']}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [14]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(raw_datasets["train"])

,question,answer,url,tags
0,"Hi Doctor, I have mild irritation in throat and runny nose since yesterday, today runny nose has improved. now I have thick phlegm/sputum with yellow/grey in color with mild cough. pls advice","It seems to be recovering\n \n Hi,\n From your description I think it will recover in 5-7 days.\nIf it does not recover, then go for antibiotics like Amoxiclav for 3- days.\n Steam inhalation/warm saline gargles will help. \nTake paracetamol for fever, if any.\n Otherwise no specific medicines are indicated for now.\n Dr Vaishalee",https://www.healthcaremagic.com/premiumquestions/How-can-mild-irritation-in-throat-and-runny-nose-along-with-thick-phlegm-and-mild-cough-be-managed/342088,None
1,I am very fair and have used dozens of sun blocks but my face always burns no matter what I do. What is the best facial sun block for fair skin?,"Physical blocker sunscreen\n \nHello\n The irritation after sunscreen use you are developing is called contact dermatitis.\nContact dermatitis after sunscreen use is not uncommon.\nThere are plenty of ingredients like fragrances, preservatives in a sunscreen that can easily cause irritation.\nMost sunscreens contains so many ingredients, its difficult to find the offending chemical. Thus the best option for you is get a special test called patch test done to identify culprit chemical.\nMost common ingredients in sunscreen which can cause irritation are PABA ( Para amino benzoic acid, benzophenones, cinnamates, octocrylene, salicylates.\nConsidering these chemicals in one form or other are used in every sunscreen, I would strictly advise you to avoid them.\nThus the best feasible option left for you is to use pure physical sun blockers. Physical blockers like zinc oxide and titanium dioxide are chemicals that prevent direct interaction of UV light and skin. Since these are large molecules they donot get absorbed inside skin and thus dont provoke any allergy or irritation and thus are safest molecules to use.\nDue to large size they leave behind whitish residue on skin and thus are not cosmetically more elegant.\nNow-a-days sunscreens with these physical blockers with micro or nano size particles are available to over come that drawback an make them cosmetically more appealing.\nIn a nutshell, try using pure physical blocker sunscreen to avoid irritation.( Eg Suncross Soft by Ranbaxy)\nHope this was useful.\nLet me know if you need any assistance from my side.\nThanks\nDr Hardik Pitroda",https://www.healthcaremagic.com/premiumquestions/What-is-the-best-facial-sun-block-for-fair-skin-while-experiencing-symptoms-of-contact-dermatitis/118640,None
2,does clonidine cause constant dizziness,"Yes it may cause dizziness.\n \n Hello,\n I can understand your concern. Yes, Dizziness is one of the side effect of Tab. Clonidine. I'd suggest you to consult with physician for dosage modification or alternative medications. Kindly do blood pressure monitoring. Kindly follow up if you have any query. take care.\n Regards,\nDr. Shyam Kale\nFamily and general physician.",https://www.healthcaremagic.com/premiumquestions/Does-Clonidine-cause-constant-dizziness/341329,None
3,"I'm having severe pain in the back (neck) area, and when I lift anything, the pain feels like a fire burning across my back (shoulders). Years ago I was told that I had two protruding discs. Could this be related?","Yes,disc protrusion is totally related to neck pain.\n \n Hello,\n Yes, disc protrusion is totally related to neck pain.\nThis is caused by the fact that the protrusion presses the nerve and cause severe pain.\nSome advice that I can give to you for relief is:\n-continue using heat or cold compresses as you are already\n-take over the counter pain killers like ibuprofen 400 mg three times a day\n-rest as much as you can\n-do stretching exercises \n-If the pain continues other options of treatment can be steroid injections or surgery.\nOf course you should discuss with your doctor about this treatment. \n Hope my answer

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [16]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [17]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [19]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [20]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them using the `text_target` parameter. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [21]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [22]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [23]:
max_input_length = 512
max_target_length = 512

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [24]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 9178, 109, 939, 912, 7893, 122, 2], [0, 118, 56, 10, 17465, 337, 18541, 204, 107, 536, 8, 67, 33, 10, 3694, 403, 9, 253, 11174, 1069, 13310, 939, 524, 129, 974, 8, 127, 675, 2294, 2198, 155, 377, 536, 117, 55, 31571, 11, 227, 1085, 939, 524, 45, 5882, 117, 464, 11, 1230, 10095, 5626, 2408, 1085, 70, 20987, 6690, 3457, 33, 57, 2430, 99, 115, 28, 1593, 19, 162, 116, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 620, 26307, 7893, 16, 59, 40, 476, 8, 145, 25781, 4, 47, 64, 912, 7385, 30, 519, 10306, 27128, 1499, 50, 27730, 9202, 1719, 3225, 11, 12777, 19, 10, 3299, 4, 1511, 41, 7530, 8474, 583, 47, 4, 17802, 47, 275, 9, 474, 2446, 2], [0, 42891, 42, 4428, 1341, 21942, 14, 528, 7, 117, 1219, 23, 42, 1046, 47, 33, 117, 4943, 13, 94, 130, 377, 4, 6690, 16, 8

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [25]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/4006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [42]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [43]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [44]:
import nltk
import numpy as np
import nltk
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [45]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/bart-base-finetuned-xsum is already a clone of https://huggingface.co/GuysTrans/bart-base-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [46]:
for i in range(0, 20):
  trainer.train()
  trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.455000,1.780504,10.367800,5.638500,9.013900,9.968600,20.000000


Upload file pytorch_model.bin:   0%|          | 1.00/532M [00:00<?, ?B/s]

Upload file runs/Aug04_04-39-51_cf569030cac8/events.out.tfevents.1691123995.cf569030cac8.376.1:   0%|         …

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   7739797..fa689dd  main -> main

   7739797..fa689dd  main -> main

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   fa689dd..005c8cf  main -> main

   fa689dd..005c8cf  main -> main



...

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   1ec06b8..3616b2f  main -> main

   1ec06b8..3616b2f  main -> main

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   3616b2f..b6e72f4  main -> main

   3616b2f..b6e72f4  main -> main



Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.165400,1.823923,10.420300,5.498600,8.977900,9.970200,20.000000


Upload file pytorch_model.bin:   0%|          | 1.00/532M [00:00<?, ?B/s]

Upload file runs/Aug04_04-39-51_cf569030cac8/events.out.tfevents.1691131885.cf569030cac8.376.10:   0%|        …

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   297ad46..f2a3c6b  main -> main

   297ad46..f2a3c6b  main -> main

To https://huggingface.co/GuysTrans/bart-base-finetuned-xsum
   f2a3c6b..8da29b2  main -> main

   f2a3c6b..8da29b2  main -> main



In [47]:
from tabulate import tabulate

In [48]:
model_before_finetuned = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [49]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["question"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=512)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [50]:
test_samples = vals_ds.select(range(8))


summaries_before_tuning = generate_summary(test_samples, model_before_finetuned)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [51]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Answer after", "Answer before"],
    )
)
print("\nTarget answers:\n")
print(
    tabulate(list(enumerate(test_samples["answer"])), headers=["Id", "Target answer"])
)
print("\nSource questions:\n")
print(tabulate(list(enumerate(test_samples["question"])), headers=["Id", "Question"]))

  Id  Answer after                                                                                                                                                                                                                                                                                                                                                                                                                                                      Answer before
----  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  --------------------------------------------------------------------------

In [52]:
summaries_after_tuning

['Kindly upload images \n \n Hello,\n I request you to kindly upload a few images of the affected area in "Reports Section" so that I am able to have a look and guide you better.\n Hope I have answered your query. Let me know if I can assist you further.',
 'Get examined by a neurologistHello,I have gone through your question and understand your concern.The headache you are having is likely to be due to sleep deprivation.The symptoms you are describing are not typical of any headache disorder.In my opinion, you should get examined by an experienced neurologist.A Head Up Tilt test may be required.Hope you found the answer helpful.Do get back to me for further information.RegardsDr N KumarNeurologist',
 "Need more informationHi dear,I have gone through your question and understand your concerns.Cough and cold are viral 95% of the times in children. They don't require antibiotics at all. In some cases, it may take up to 2 weeks for complete recovery. You need to consult a doctor and get e

In [53]:
list(enumerate(test_samples["answer"]))

[(0,
  'Acute Urticaria Hello. Thank you. I am Dr kakkar (Dermatologist and Venereologist). I have noted your concern. You seem to have Hives/ Urticaria. Acute Urticaria can happen as an allergic reaction to certain foods or food preservatives. I suggest you to apply a soothing lotion e.g Calamine lotion. In addition I suggest you to take an oral antihistamine e.g Cetrizine 10mg tab once a day. Regards'),
 (1,
  'See a neurologist\n \n Hi,\n Thanks for posting your query.\n So you have a sun-exposure headache. It could be hyperthermia (sunstroke), electrolyte imbalance, photosensitivity of eyes or it maybe a migraine. \n If sun is causing headaches, then best cure is to NOT step out in sun. Stay in cool, shady place. If mild painkiller like ibuprofen 200 mg makes the headache better, then continue with it. Otherwise stronger painkillers will be needed. Take ORS after exposure to sun or excessive sweating.\n A CT Scan must be done. See a neurologist for further management.\n Hope it hel

In [54]:
# from transformers import pipeline, Conversation

# chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)
# conversation = Conversation("hello am 22 years old and i have type 2diabet i wanted to sign for a new gym which use electric vibes to improve heart pulses and fasten the process of loosing weight i wanted to know if it is dangerous for me knowing that 20min of this sports equals 4 h of normal one thanks	")
# conversation = chatbot(conversation)
# conversation.generated_responses[-1]

You can now upload the result of the training to the Hub, just execute this instruction:

In [55]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```